In [1]:
# Import GoldUtilities
push!(LOAD_PATH, joinpath(@__DIR__, "..", "scripts"))
using GoldUtilities
import GoldUtilities.Loader as loader

In [3]:
# Add the MSSim package
using Pkg
Pkg.add(url="https://github.com/euriqa-brassboard/MSSim.jl")
using MSSim
const Opts = MSSim.Optimizers
const SS = MSSim.SegSeq
const SL = MSSim.SymLinear
const Seq = MSSim.Sequence

    Updating git-repo `https://github.com/euriqa-brassboard/MSSim.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


MSSim.Sequence

In [2]:
params = loader.GoldParams("../system params/072125_goldparams_13ions.json")

GoldUtilities.Loader.GoldParams([2.29306, 2.31375, 2.33556, 2.35784, 2.3805, 2.40237, 2.42394, 2.44446, 2.46356, 2.4808, 2.49555, 2.50966, 2.51611], [0.12582898125060124, 0.1254757153730443, 0.1250569316248486, 0.12461075061627393, 0.12415467122300929, 0.12370104825133979, 0.12325955010900741, 0.12283842408236283, 0.12244532009242795, 0.12208804746084248, 0.12177563418463196, 0.12151470773707744, 0.1213753723354335], [0.002163619698947626 -0.019894333993010195 … -0.019894333993005258 0.0021636196989478743; 0.0071396181636678106 -0.05753283102846898 … 0.05753283102846495 -0.007139618163669534; … ; 0.42615110225953595 0.36754442103650975 … -0.36754442103645346 -0.4261511022594881; 0.2773500981126128 0.2773500981125969 … 0.27735009811265454 0.27735009811265965], Dict("x1" => 0.0, "x4" => 0.00131, "x2" => 0.0124, "qzy" => 0.12, "dy" => 71.0, "qzz" => 0.85, "x3" => 0.0))

In [ ]:
# Construct modes matrtix using system parameters
const modes = Seq.Modes()
const ion1 = 2
const ion2 = 6
for i in 1:13
    push!(modes, 2π * params.radial_modes[i], params.participation_factors[i, ion1] * params.participation_factors[i, ion2] * params.etas[i]^2)
end

In [ ]:
# Function to give a balckman amplitude profile
function blackman_start_end(x)
    if -0.5 <= x <= 0.5
        return 1.0
    elseif x < 0
        x = (x + 0.5) * 2
    else
        x = (x - 0.5) * 2
    end
    return 0.42 + 0.5 * cospi(x) + 0.08 * cospi(2 * x)
end

blackman_start_end (generic function with 1 method)

In [7]:
const nseg = 120
const buf_opt = SL.ComputeBuffer{nseg,Float64}(Val(SS.mask_allδ), Val(SS.mask_allδ));
const buf_plot = SL.ComputeBuffer{nseg,Float64}(Val(SS.mask_full), Val(SS.mask_full));

In [8]:
function _objfunc(vals)
    dis = vals[1]
    disδ = vals[2]
    area = vals[3]
    areaδ = vals[4]
    τ = vals[5]

    return 5 * dis + disδ / 100 + (abs(area) - π / 2)^2 * 100 + (areaδ / 1e4)^2
end
const objfunc = Opts.autodiff(_objfunc);

In [9]:
const nlmodel = Seq.Objective(SL.pmask_full,
                              ((:dis2, 0), (:disδ2, 0), (:area, 0),
                               (:areaδ, 0), (:τ, 0)),
                              objfunc, modes, buf_opt,
                              freq=Seq.FreqSpec(true, sym=false),
                              amp=Seq.AmpSpec(cb=blackman_start_end));

In [ ]:
const nargs = Seq.nparams(nlmodel)
const tracker = Opts.NLVarTracker(nargs)
Opts.set_bound!(tracker, nlmodel.param.τ, 0.1, 4)
Opts.set_bound!(tracker, nlmodel.param.Ωs[1], 0.1, 0.2)
for ω in nlmodel.param.ωs
    Opts.set_bound!(tracker, ω, 2π * 2.39, 2π * 2.52)
end

Base.Meta.ParseError: ParseError:
# Error @ /home/bahaa/Documents/WS/GoldGates.jl/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:6:54
for ω in nlmodel.param.ωs
    Opts.set_bound!(tracker, ω, 2π * 2.39, 2π * 2.52)
#                                                    └ ── Expected `end`

In [11]:
# const opt = NLopt.Opt(:LD_SLSQP, nargs)
const opt = NLopt.Opt(:LD_LBFGS, nargs)
# NLopt.xtol_rel!(opt, 1e-5)
# NLopt.ftol_rel!(opt, 1e-5)
NLopt.min_objective!(opt, nlmodel)
NLopt.lower_bounds!(opt, Opts.lower_bounds(tracker))
NLopt.upper_bounds!(opt, Opts.upper_bounds(tracker))
NLopt.maxtime!(opt, 10)

UndefVarError: UndefVarError: `NLopt` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: NLopt is loaded but not imported in the active module Main.

In [ ]:
best_obj = 100.0
best_params = nothing
@time for i in 1:50
    global best_obj, best_params
    obj, params, ret = @time NLopt.optimize(opt, Opts.init_vars!(tracker))
    if getfield(NLopt, ret) < 0
        continue
    end
    if obj < best_obj
        best_obj = obj
        area = nlmodel(Val((:area, 0)), params)
        best_status = (
            obj = obj,
            dis = nlmodel(Val((:dis2, 0)), params),
            disδ = nlmodel(Val((:disδ2, 0)), params),
            area = area,
            areaε = abs(area) - π / 2,
            areaδ = nlmodel(Val((:areaδ, 0)), params),
            total_t = nlmodel(Val((:τ, 0)), params),
            Ωmax = params[nlmodel.param.Ωs[1]],
        )
        println(best_status)
        best_params = params
    end
end